# 1) Sentinel-2 image search

    This code searches for and visualizes satellite images within specified geographical boundaries and time frame using the EODataAccessGateway from the eodag library, filters by cloud cover, and then displays the search results and area boundary on an interactive map using Folium.

In [ ]:
from eodag import EODataAccessGateway
import os
import rasterio  
import numpy as np   
import folium  
import rasterio.mask
import geopandas as gpd
from osgeo import gdal
import gc
import rasterio
from memory_profiler import memory_usage

shapefile_path = '/path/to/area_boundaries/boundaries.shp'

footprint_data = gpd.read_file(shapefile_path)
coordinates = footprint_data.total_bounds

min_x = coordinates[0]
min_y = coordinates[1]
max_x = coordinates[2]
max_y = coordinates[3]

dag = EODataAccessGateway()
product_type = "S2_MSI_L2A"

footprint = [min_x, min_y, max_x, max_y]
print('min_x:', min_x, 'min_y', min_y, 'max_x:', max_x, 'max_y:', max_y )

cloud_cover = 5
start = "2022-12-24"
end =   "2022-12-27"

search_results, estimated_total_nbr_of_results = dag.search(
    productType=product_type,
    geom=footprint,
    start=start,
    end=end,
    cloudCover=cloud_cover,
)

print("Estimated total number of available images:", estimated_total_nbr_of_results)
print('Number of searched images: :', len(search_results))

if len(search_results) > 0:
    emap = folium.Map([7, 3], zoom_start=7)
    layer = folium.features.GeoJson(
        data=search_results.as_geojson_object(),
        tooltip=folium.GeoJsonTooltip(fields=["title"]),).add_to(emap)
    
    shapefile_geojson = footprint_data.__geo_interface__
    folium.GeoJson(shapefile_geojson).add_to(emap)

    emap
else:
    print("No search results found.")

emap

# 2) Calculation of spectral indices

    This code is designed to process satellite imagery, specifically by finding and organizing image files by their resolution and spectral bands, calculating various vegetation and water indices from these images, and saving the results as TIFF files in a specified output directory.

In [ ]:
import os
import numpy as np
import rasterio
from rasterio.windows import Window
from rasterio.enums import Resampling
import gc
from memory_profiler import memory_usage
from rasterio.transform import from_origin
from rasterio.plot import show
from datetime import datetime

output_folder = '/path/to/save/indices/'

img_paths = []

for product in search_results:
    properties = product.properties

    img_path = '/initial_folder_for_image_search/ + properties["productIdentifier"]
    img_date = properties["beginningDateTime"][:10] 
    img_paths.append(img_path)

all_data_dict = {} 
index_results_by_tile = {}

def find_jp2_files(img_path):
    jp2_files_by_resolution = {"R20m": {"B02": None, "B03": None, "B04": None, "B8A": None, "B11": None, "B12": None}}
    for root, _, files in os.walk(img_path):
        for file in files:
            if file.lower().endswith(".jp2"):
                if "R20m" in root and "B02" in file:
                    jp2_files_by_resolution["R20m"]["B02"] = os.path.join(root, file)
                elif "R20m" in root and "B03" in file:
                    jp2_files_by_resolution["R20m"]["B03"] = os.path.join(root, file)
                elif "R20m" in root and "B04" in file:
                    jp2_files_by_resolution["R20m"]["B04"] = os.path.join(root, file)
                elif "R20m" in root and "B8A" in file:
                    jp2_files_by_resolution["R20m"]["B8A"] = os.path.join(root, file)
                elif "R20m" in root and "B11" in file:
                    jp2_files_by_resolution["R20m"]["B11"] = os.path.join(root, file)
                elif "R20m" in root and "B12" in file:
                    jp2_files_by_resolution["R20m"]["B12"] = os.path.join(root, file)
                    
    return jp2_files_by_resolution

def divide_by_bands(jp2_files):
    """
    Function for grouping JP2 files by bands (B02, B03, B04, B8A, B11, B12) for a given resolution.
    """
    bands = ["B02", "B03", "B04", "B8A", "B11", "B12"]  
    divided_lists = {}
    for resolution, files in jp2_files.items():
        divided_lists[resolution] = {}
        for band in bands:
            if band in files and files[band] is not None:
                divided_lists[resolution][band] = files[band]
    return divided_lists

def calculate_all_indices(output_folder_tile, red_path, nir_path, green_path, blue_path, swir1_path, swir2_path):
    with rasterio.open(red_path) as red_src, rasterio.open(nir_path) as nir_src, rasterio.open(green_path) as green_src, rasterio.open(blue_path) as blue_src, rasterio.open(swir1_path) as swir1_src, rasterio.open(swir2_path) as swir2_src:
        red = red_src.read(1)
        nir = nir_src.read(1)
        green = green_src.read(1)
        blue = blue_src.read(1)
        swir1 = swir1_src.read(1)
        swir2 = swir2_src.read(1)
    
    # Normalization and conversion of 0 to NaN
    red = red.astype(np.float32) / 65535.0
    nir = nir.astype(np.float32) / 65535.0
    green = green.astype(np.float32) / 65535.0
    blue = blue.astype(np.float32) / 65535.0
    swir1 = swir1.astype(np.float32) / 65535.0
    swir2 = swir2.astype(np.float32) / 65535.0

    np.seterr(divide='ignore', invalid='ignore')

    red[red == 0] = np.nan
    nir[nir == 0] = np.nan
    green[green == 0] = np.nan
    blue[blue == 0] = np.nan
    swir1[swir1 == 0] = np.nan
    swir2[swir2 == 0] = np.nan    
    
    ndvi = (nir - red) / (nir + red)
    gndvi = (nir - green) / (nir + green)
    evi = 2.5 * ((nir - red) / (nir + 6 * red - 7.5 * blue + 1))
    savi = (1 + 0.5) * ((nir - red) / (nir + red + 0.5)) 
    osavi = ((nir - red) / (nir + red + 0.16))
    dvi = nir - red
    sr = red / nir
    gemi = ((2 * (nir ** 2 - red ** 2) + 1.5 * nir + 0.5 * red) / (nir + red + 0.5)) * (1 - 0.25 * ((2 * (nir ** 2 - red ** 2) + 1.5 * nir + 0.5 * red) / (nir + red + 0.5))) - ((red - 0.125) / (1 - nir))
    ndwi = (green - nir) / (green + nir)
    mndwi = (green - swir1) / (green + swir1)
    lswi = (nir - swir1) / (nir + swir1)
    ui = (swir2 - nir) / (swir2 + nir)
    ndbi = (swir1 - nir) / (swir1 + nir)
    mndbi = (swir2 - blue) / (swir2 + blue)      
    bsi = ((red + swir1) - (nir + blue)) / ((red + swir1) + (nir + blue))                                                                          

    indices = {"NDVI": ndvi, "GNDVI": gndvi, "EVI": evi, "SAVI": savi, "OSAVI": osavi, "DVI": dvi, "SR": sr, "GEMI": gemi, "NDWI": ndwi, "BSI": bsi, "MNDBI": mndbi, "LSWI": lswi, "MNDWI": mndwi, "NDBI": ndbi, "UI": ui,}
    
    img_tile = img_path[-27:-20]
    img_date = img_path[-20:-12]
    form_date = f"{img_date[:4]}-{img_date[4:6]}-{img_date[6:]}"
    
    # Save calculated indexes to tiff files
    for index_name, index_data in indices.items():
        output_file = os.path.join(output_folder_tile, f"S2_{index_name}_{form_date}_{img_tile}.tif")

        with rasterio.open(output_file, 'w', driver='GTiff', width=red.shape[1], height=red.shape[0], count=1, dtype=str(index_data.dtype), crs=red_src.crs, transform=red_src.transform) as dst:
            dst.write(index_data, 1)
            
def create_folder_if_not_exists(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path) 

def process_product(img_path):
    properties = product.properties
    img_date = properties["beginningDateTime"][:10] 
    
    print(f"Image processing: {img_path}\n")
    
    jp2_files_by_resolution = find_jp2_files(img_path)
    divided_lists_by_bands = divide_by_bands(jp2_files_by_resolution)
    all_data_dict[img_path] = divided_lists_by_bands
    
    tile_identifier = img_path.split("/")[-1].split("_")[-2]
    
    folder_name = img_path[-27:-5]
    output_folder_tile = os.path.join(output_folder, folder_name) 
    
    create_folder_if_not_exists(output_folder_tile)  
 
    red_path = divided_lists_by_bands["R20m"]["B04"]
    nir_path = divided_lists_by_bands["R20m"]["B8A"]
    green_path = divided_lists_by_bands["R20m"]["B03"]
    blue_path = divided_lists_by_bands["R20m"]["B02"]
    swir1_path = divided_lists_by_bands["R20m"]["B11"]
    swir2_path = divided_lists_by_bands["R20m"]["B12"]
    
    with rasterio.open(red_path) as src_red:
        profile = src_red.profile
    
    if all([red_path, nir_path, green_path, blue_path, swir1_path, swir2_path]):
        index_results = calculate_all_indices(output_folder_tile, red_path, nir_path, green_path, blue_path, swir1_path, swir2_path)
                   
gc.collect()

for img_path in img_paths:
    folder_name = img_path[-27:-5]
    output_folder_tile = os.path.join(output_folder, folder_name)
    create_folder_if_not_exists(output_folder_tile)
    process_product(img_path)
                                            
print()
print("All files saved :)")